## VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
weather_csv = pd.read_csv("../WeatherPy/Output/cities.csv")
weather_csv.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Zheleznodorozhnyy,55.7503,38.0042,6.80,78,90,15.66,RU,1613178286
1,1,São João da Barra,-21.6403,-41.0511,77.41,82,100,14.63,BR,1613178287
2,2,Hilo,19.7297,-155.0900,82.40,51,20,5.75,US,1613178287
3,3,Rikitea,-23.1203,-134.9692,80.53,72,26,10.11,PF,1613178287
4,4,Ponta do Sol,32.6667,-17.1000,57.00,74,0,5.01,PT,1613178287


## Humidity Heatmap

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_csv[["Lat", "Lng"]]

humidity = weather_csv["Humidity"]

In [5]:
# Create a humidity Heatmap layer
weather_heatmap = gmaps.figure(center=(0,0), zoom_level = 1.5)

humidity_layer = gmaps.heatmap_layer(locations, weights= humidity, 
                                 dissipating= False, max_intensity=100,
                                 point_radius = 1)

weather_heatmap.add_layer(humidity_layer)

weather_heatmap

Figure(layout=FigureLayout(height='420px'))

## Ideal Vacation Weather

In [13]:
weather_csv = weather_csv.loc[weather_csv["Max Temp"] <= 80, :]
weather_csv = weather_csv.loc[weather_csv["Max Temp"] >= 70, :]
weather_csv = weather_csv.loc[weather_csv["Wind Speed"] <= 6.8, :]
weather_csv = weather_csv.loc[weather_csv["Cloudiness"] == 0, :]
weather_csv

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Paraná,-31.7320,-60.5238,77.00,82,0,4.70,AR,1613178288
9,9,Busselton,-33.6500,115.3333,70.00,65,0,3.00,AU,1613178288
69,69,Douentza,14.9951,-2.9517,70.59,20,0,2.80,ML,1613178302
78,78,Indiaroba,-11.5192,-37.5117,76.41,79,0,5.23,BR,1613178305
164,164,Bonthe,7.5264,-12.5050,77.85,86,0,5.66,SL,1613178320
173,173,Cap Malheureux,-19.9842,57.6142,73.99,81,0,3.00,MU,1613178321
201,201,Goundam,16.4145,-3.6708,73.44,18,0,6.64,ML,1613178326
391,391,Santa Isabel Rio Negro,-0.4139,-65.0192,78.06,83,0,3.06,BR,1613178360
439,439,Ladário,-19.0047,-57.6017,73.35,91,0,2.17,BR,1613178368
484,484,Malakal,9.5334,31.6605,73.67,21,0,5.30,SS,1613178377


## Hotel Map

In [15]:
hotel_df = weather_csv
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,Paraná,-31.7320,-60.5238,77.00,82,0,4.70,AR,1613178288,
9,9,Busselton,-33.6500,115.3333,70.00,65,0,3.00,AU,1613178288,
69,69,Douentza,14.9951,-2.9517,70.59,20,0,2.80,ML,1613178302,
78,78,Indiaroba,-11.5192,-37.5117,76.41,79,0,5.23,BR,1613178305,
164,164,Bonthe,7.5264,-12.5050,77.85,86,0,5.66,SL,1613178320,
173,173,Cap Malheureux,-19.9842,57.6142,73.99,81,0,3.00,MU,1613178321,
201,201,Goundam,16.4145,-3.6708,73.44,18,0,6.64,ML,1613178326,
391,391,Santa Isabel Rio Negro,-0.4139,-65.0192,78.06,83,0,3.06,BR,1613178360,
439,439,Ladário,-19.0047,-57.6017,73.35,91,0,2.17,BR,1613178368,
484,484,Malakal,9.5334,31.6605,73.67,21,0,5.30,SS,1613178377,


In [23]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#    print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        print("Hotel found")
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Hotel found
Hotel found
Missing field/result... skipping.
Hotel found
Hotel found
Hotel found
Missing field/result... skipping.
Hotel found
Hotel found
Hotel found


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,Paraná,-31.7320,-60.5238,77.00,82,0,4.70,AR,1613178288,InterTower Hotel
9,9,Busselton,-33.6500,115.3333,70.00,65,0,3.00,AU,1613178288,Pullman Bunker Bay Resort Margaret River Region
69,69,Douentza,14.9951,-2.9517,70.59,20,0,2.80,ML,1613178302,
78,78,Indiaroba,-11.5192,-37.5117,76.41,79,0,5.23,BR,1613178305,Pousada O Forte
164,164,Bonthe,7.5264,-12.5050,77.85,86,0,5.66,SL,1613178320,Bonthe Holiday Village
173,173,Cap Malheureux,-19.9842,57.6142,73.99,81,0,3.00,MU,1613178321,Mauricia Beachcomber Resort & Spa
201,201,Goundam,16.4145,-3.6708,73.44,18,0,6.64,ML,1613178326,
391,391,Santa Isabel Rio Negro,-0.4139,-65.0192,78.06,83,0,3.06,BR,1613178360,Hotel Ayuana
439,439,Ladário,-19.0047,-57.6017,73.35,91,0,2.17,BR,1613178368,Nacional Palace Hotel
484,484,Malakal,9.5334,31.6605,73.67,21,0,5.30,SS,1613178377,Nile Palace Hotel


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

weather_heatmap.add_layer(hotel_layer)

# Display figure
weather_heatmap

Figure(layout=FigureLayout(height='420px'))